# 2.2 EconML：估计 CATE/个体化治疗效果（以 UCI Heart 数据为例）

本 Notebook 演示如何用 **EconML** 的 **T-learner**（亦可替换为 X-learner 等）估计个体化治疗效果：
1) 加载数据并构造示例治疗/结局/协变量
2) 训练 T-learner 并输出每个个体的 CATE
3) 可视化 CATE 分布、分位数对比

---

### 环境准备（如已安装可跳过）
```bash
pip install -U econml scikit-learn pandas matplotlib numpy
```
> 说明：此示例仅用于方法演示，变量赋义是“教学假设”。实际研究需依据临床机制构建 DAG/变量。

In [ ]:
# ① 加载 UCI Heart Disease 数据
import pandas as pd

url = 'https://raw.githubusercontent.com/selva86/datasets/master/Heart.csv'
df = pd.read_csv(url)
print(df.head())
print('\nColumns:', list(df.columns))


**教学假设（仅示例）**：

- `T`（treatment）= 是否存在心脏病 `AHD`（Yes/No → 1/0）；
- `Y`（outcome）= `MaxHR`（最大心率，用作示例结局）；
- `X`（协变量）= `Age, Sex, Chol, RestBP` 等基线特征。

In [ ]:
# ② 训练 T-learner / 输出 CATE
from econml.metalearners import TLearner
from sklearn.ensemble import RandomForestRegressor
import numpy as np

X = df[['Age','Sex','Chol','RestBP']].copy()
T = (df['AHD']=='Yes').astype(int).values
Y = df['MaxHR'].values

t_learner = TLearner(models=RandomForestRegressor())
t_learner.fit(Y, T, X=X)
cate = t_learner.effect(X)
print('前 10 个个体的 CATE：', np.round(cate[:10], 3))


In [ ]:
# ③ 可视化：CATE 分布 + 分位数描述
import matplotlib.pyplot as plt
import numpy as np
plt.figure()
plt.hist(cate, bins=30)
plt.xlabel('Estimated CATE')
plt.ylabel('Count')
plt.title('Distribution of Individual Treatment Effects (CATE)')
plt.show()

q = np.quantile(cate, [0.1, 0.5, 0.9])
print('CATE 分位数 (10%, 50%, 90%):', q)


**延伸**：
- 将 `TLearner` 换成 `XLearner`、`DRLearner`、`CausalForestDML` 等；
- 在 `econml` 中加入更多协变量/非线性项，并结合交互做 **异质性治疗效果（HTE）** 分析；
- 回到医学场景：明确治疗机制、时间顺序，先画 DAG 再建模。